
# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shap

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor

from statistics import mean
from scipy.stats import tmean
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.preprocessing import LabelEncoder
from xgboost import plot_importance
from sklearn.inspection import plot_partial_dependence

In [3]:
# loading csv
df_original = pd.read_csv('data/final_dataset.csv').drop("Unnamed: 0", axis=1)

df = df_original.copy()

Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.


In [4]:
# df_original[df_original["Name"] == "Mohamed-Salah"].groupby(["Pos"]).mean()["Total Points"]

In [5]:
df = df.rename(columns={'GW_x': 'GW'})
# df = df[~(df['position'].isin(['GK']))]
df = df[df['position'].isin(['FWD'])]

In [6]:
df_original[["expected_goals", "xG"]]

,expected_goals,xG
0,NaN,0.1
1,NaN,0.1
2,NaN,0.0
3,NaN,0.1
4,NaN,0.0
...,...,...
24678,0.00,0.0
24679,0.00,0.0
24680,0.00,0.0
24681,0.00,0.0


In [7]:
df_original["goals_conceded"].unique()

array([ 2.,  1.,  0., nan,  3.,  4.,  5.,  7.,  6.,  8.,  9.])

In [8]:
# df_original.columns.to_list()

In [9]:
df["position"].unique()

array(['FWD'], dtype=object)

In [10]:
df.shape

(2589, 117)

In [11]:
to_predict = ["total_points"]
features1 = ["Was Home", "Team rating", "Opp rating", 
             # 'ScoreForLast5', 'ScoreAgainstLast5',
             ]
features_to_roll = [
                "Min", "Start", # time played
                'Gls', 'Sh', 'SoT', # Goals
                'Ast', # Assists
                'CrdY', 'CrdR', # Cards
                "xG", 'xAG', # Expected
                'CS', # Defence
                'Team Score', 'Opp Score', # Team form
                # "xGPoints", "CSPoints", # Position-scaled
                # "Cmp%", "PrgP", "PrgC", "Succ", "PKwon",
                'bonus', 'bps', # Bonus
                'influence', 'creativity', 'threat', 'ict_index', #ICT
                "total_points" # FPL points
            ]
features_to_roll = "Min"
info = ["Name", "GW", "Season", "Squad", "Opponent", "Was Home", "Team rating", "Opp rating", "position"]

In [12]:
# df[features_to_roll + info].isnull().sum(axis=0).sort_values(ascending=False).head(30)

# Preparing dataset

In [13]:
# features = ["Was Home", "Team rating", "Opp rating", 'ScoreForLast5', 'ScoreAgainstLast5', 
#         'MinAvgOverall', 'GlsAvgOverall',
#        'AstAvgOverall', 'CrdYAvgOverall', 'CrdRAvgOverall', 'xGAvgOverall',
#        'xAGAvgOverall', 'CSAvgOverall', 'Team ScoreAvgOverall',
#        'Opp ScoreAvgOverall']

In [14]:
def calculate_xg_points(row):
    if row["position"] == "GK":
        return row["xG"] * 6
    elif row["position"] == "DEF":
        return row["xG"] * 6
    elif row["position"] == "MID":
        return row["xG"] * 5
    elif row["position"] == "FWD":
        return row["xG"] * 4
    else:
        return row["xG"] * 5

df["xGPoints"] = df.apply(calculate_xg_points, axis=1)

In [15]:
def calculate_cs_points(row):
    if row["position"] == "GK":
        return row["xG"] * 4
    elif row["position"] == "DEF":
        return row["xG"] * 4
    elif row["position"] == "MID":
        return row["xG"] * 1
    elif row["position"] == "FWD":
        return row["xG"] * 0
    else:
        return row["xG"] * 1

df["CSPoints"] = df.apply(calculate_cs_points, axis=1)

In [16]:
df.shape

(2589, 119)

In [17]:
def add_rolling_features(df, features1, features_to_roll):
    rolling_gameweeks = [2, 5]
    rolling_gameweeks = [2]
    features = features1
    
    for r in rolling_gameweeks:
        form_means = df.groupby(["Name"])[features_to_roll].rolling(r, min_periods=1).mean().groupby(["Name"]).shift(1).reset_index()
        # form_means = df.groupby("Name")[features_to_roll].rolling(window=r, min_periods=1).mean().shift(1).reset_index()
        # df['Min_2'] = df.groupby('Name')['Min'].transform(lambda x: x.rolling(2, 1).mean())
        # form_means = pd.DataFrame( df[features_to_roll].groupby(df['Name']).shift(1).rolling(r, min_periods=1).mean() )
        # print(form_means.isna().sum())
        # form_means = form_means.shift(1)
        form_means = form_means.fillna(method='bfill')
        
        form_means.columns = [f'{col}{"_"}{r}' for col in form_means.columns]
        features += form_means.columns.tolist()
        features = list(filter(lambda x: x not in ["Name_" + str(r)], features))
        df = pd.concat([df, form_means], axis=1)
        # features += ["Min_2"]
        print(r, df.shape)
        
    return df, features

In [18]:
def ohe(df, ohe_columns, features):
    # one hot encoding
    for c in ohe_columns:
        ohe_c = pd.get_dummies(df[c], dtype="int64")
        df = pd.concat([df, ohe_c], axis=1)
        # df = df.drop(c, axis=1)

        features += ohe_c.columns.tolist()
        
    return df, features

In [19]:
def label_encoding(df, column_to_encode):
    
    mapping_dict = {
        'Y': 2,
        'Y*': 1,
        'N': 0,
    }
    
    df[column_to_encode] = df[column_to_encode].map(mapping_dict)
    
    return df

In [20]:
df = df.reset_index()

In [21]:
# df.head(30)

In [22]:
df, features = add_rolling_features(df, features1, features_to_roll)

2 (2589, 123)


In [23]:
df, features = ohe(df, ["position"], features)

In [24]:
df = label_encoding(df, "Start")

In [25]:
df = df[np.unique(["Min"] + features + info + to_predict)]

In [26]:
df = df[pd.to_numeric(df["GW"], errors="coerce").notna()]
df["GW"] = df["GW"].astype("uint64")

In [63]:
# df.head(90).tail(30)

In [64]:
# df.tail(40)

In [29]:
# dropping NaNs
df = df.dropna(axis=0)

In [30]:
df = df.set_index(["Name", "Season", "GW"], drop=False)

In [31]:
df.columns

Index(['FWD', 'GW', 'Min', 'Min_2', 'Name', 'Opp rating', 'Opponent', 'Season',
       'Squad', 'Team rating', 'Was Home', 'level_1_2', 'position',
       'total_points'],
      dtype='object')

In [32]:
df.shape

(2589, 14)

# Choosing features and splitting data

In [33]:
GAMEWEEK_TO_PREDICT = 31
SEASON_TO_PREDICT = "2022-23"

In [34]:
# gameweeks 6-14 is training data, gw 15 is test data
X_train = df[((df["Season"] <= SEASON_TO_PREDICT) & (df["GW"] >= 1) & (df["GW"] <= GAMEWEEK_TO_PREDICT - 1))][features].reset_index(drop=True)
y_train = df[((df["Season"] <= SEASON_TO_PREDICT) & (df["GW"] >= 1) & (df["GW"] <= GAMEWEEK_TO_PREDICT - 1))][to_predict].reset_index(drop=True)

In [35]:
X_test = df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT) ][features].reset_index(drop=True)
y_test = df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT) ][to_predict].reset_index(drop=True)

In [36]:
X_train.shape, X_test.shape

((2045, 6), (44, 6))

# Model

In [37]:
class HybridModel:
    def __init__(self, models, names, weights):
        self.models = models
        self.model_names = names
        self.weights = weights
    
    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
        
    def predict(self, X):
        model_preds = [model.predict(X) for model in self.models]
        preds = []
        for i in range(len(model_preds[0])):
            preds.append(np.mean([x[i] for x in model_preds]))
        return preds
    
    def evaluate(self, X, y):
        ys = [np.expm1(model.predict(X)) for model in self.models]
        y_ensemble = self.predict(X)
        
        rmsle_ys = [(mean_squared_log_error(y, y_model) ** 0.5) for y_model in ys]
        rmsle_ensemble = mean_squared_log_error(y, y_ensemble) ** 0.5
        
        for i in range(0, len(self.models)):
            print(f"RMSLE on {self.model_names[i]} model: {rmsle_ys[i]}")
        print(f"RMSLE on ensemble model: {rmsle_ensemble}")

In [38]:
lgbm = LGBMRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
gbr = GradientBoostingRegressor(random_state=42)

names = [
    "LGBM",
    "XGB",
    "GBR"
]

weights = [
    1,
    1,
    1
]

# hybrid arguments
models = [
    lgbm, 
    xgb, 
    gbr
]

model = HybridModel(models, names, weights)

In [39]:
# model = GradientBoostingRegressor(random_state=42)
# model = LGBMRegressor(random_state=42)
# model = XGBRegressor(random_state=42)
# model = RandomForestRegressor()

In [40]:
%%time
model.fit(X_train, np.array(y_train).ravel())

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000162 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 941
[LightGBM] [Info] Number of data points in the train set: 2045, number of used features: 5
[LightGBM] [Info] Start training from score 2.609291
CPU times: total: 1.31 s
Wall time: 516 ms


# Evaluation

In [41]:
# make predictions on the test data
predictions = model.predict(X_test)

In [42]:
def pairwise_accuracy(predicted_scores, true_scores):
    if len(predicted_scores) != len(true_scores):
        raise ValueError("The length of predicted_scores and true_scores must be the same.")

    num_pairs = 0
    num_correct_pairs = 0

    for i in range(len(predicted_scores)):
        for j in range(i + 1, len(predicted_scores)):
            # Check if the predicted order matches the true order
            if (predicted_scores[i] > predicted_scores[j] and true_scores[i] > true_scores[j]) or \
               (predicted_scores[i] < predicted_scores[j] and true_scores[i] < true_scores[j]):
                num_correct_pairs += 1
            num_pairs += 1

    pairwise_accuracy = num_correct_pairs / num_pairs
    return pairwise_accuracy

In [43]:
df_predictions = df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT)].reset_index(drop=True)
df_predictions.loc[:, "pred"] = predictions

In [44]:
def get_top_performer_names(df, no_top, no_gws):
    return df.reset_index(drop=True).groupby("Name")["total_points"].mean().groupby("Name").tail(no_gws).sort_values(ascending=False).head(no_top).index.to_list()

In [45]:
def pairwise_accuracy_topX(model, df, top_x):
    top_performers = df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name"].isin(get_top_performer_names(df, top_x, 50)))]
    X = top_performers[features].reset_index(drop=True)
    y = np.array(top_performers[to_predict].reset_index(drop=True))
    preds = model.predict(X)
    # print(preds)
    # print(y)
    
    return  pairwise_accuracy(preds, y)

In [46]:
# df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name"].isin(get_top_performer_names(df, 20, 50)))].columns

In [47]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

pairwise_acc = pairwise_accuracy(np.array(y_test), predictions)
pairwise_accuracy_top20 = pairwise_accuracy_topX(model, df, 20)
pairwise_accuracy_top100 = pairwise_accuracy_topX(model, df, 100)

In [48]:
print("MAE:", mae)
print("MSE:", mse)
print("Pairwise accuracy:", pairwise_acc)
print("Pairwise accuracy @TOP100:", pairwise_accuracy_top100)
print("Pairwise accuracy @TOP20:", pairwise_accuracy_top20)

MAE: 2.6379185035198383
MSE: 17.102003797359263
Pairwise accuracy: 0.5486257928118393
Pairwise accuracy @TOP100: 0.5486257928118393
Pairwise accuracy @TOP20: 0.5147058823529411


In [49]:
# df[df["Name"] == "Abdoulaye-Doucoure"][info]

In [50]:
# model.predict(df[df["GW"] == GAMEWEEK_TO_PREDICT][df["Name"] == "Mohamed-Salah"][features])

In [51]:
preds = df_predictions[info + to_predict + ["pred"]].sort_values(by=["pred"], ascending = False)

In [52]:
preds.head(30)

,Name,GW,Season,Squad,Opponent,Was Home,Team rating,Opp rating,position,total_points,pred
17,Erling-Haaland,31,2022-23,Manchester City,Leicester City,1.0,2063.473633,1725.040527,FWD,12.0,6.965257
21,Harry-Kane,31,2022-23,Tottenham,Bournemouth,1.0,1854.798462,1668.258545,FWD,2.0,6.159525
22,Ivan-Toney,31,2022-23,Brentford,Wolves,0.0,1785.234741,1720.871338,FWD,2.0,6.032191
5,Callum-Wilson,31,2022-23,Newcastle Utd,Aston Villa,0.0,1879.303101,1804.471680,FWD,1.0,5.738616
33,Ollie-Watkins,31,2022-23,Aston Villa,Newcastle Utd,1.0,1804.471680,1879.303101,FWD,16.0,5.456153
0,Alexander-Isak,31,2022-23,Newcastle Utd,Aston Villa,0.0,1879.303101,1804.471680,FWD,2.0,5.115699
19,Gabriel-Jesus,31,2022-23,Arsenal,West Ham,0.0,1957.524048,1754.553955,FWD,7.0,3.957874
6,Carlos-Vinicius,31,2022-23,Fulham,Everton,0.0,1713.854858,1690.919678,FWD,1.0,3.514711
3,Brennan-Johnson,31,2022-23,Nott'ham Forest,Manchester Utd,1.0,1638.596069,1882.111816,FWD,2.0,3.403676
14,Eddie-Nketiah,31,2022-23,Arsenal,West Ham,0.0,1957.524048,1754.553955,FWD,1.0,3.363196


In [53]:
preds[preds["Squad"] == "Manchester City"].head(20)

,Name,GW,Season,Squad,Opponent,Was Home,Team rating,Opp rating,position,total_points,pred
17,Erling-Haaland,31,2022-23,Manchester City,Leicester City,1.0,2063.473633,1725.040527,FWD,12.0,6.965257
26,Julian-Alvarez,31,2022-23,Manchester City,Leicester City,1.0,2063.473633,1725.040527,FWD,1.0,1.545390


In [54]:
preds[preds["position"] == "FWD"].head(10)

,Name,GW,Season,Squad,Opponent,Was Home,Team rating,Opp rating,position,total_points,pred
17,Erling-Haaland,31,2022-23,Manchester City,Leicester City,1.0,2063.473633,1725.040527,FWD,12.0,6.965257
21,Harry-Kane,31,2022-23,Tottenham,Bournemouth,1.0,1854.798462,1668.258545,FWD,2.0,6.159525
22,Ivan-Toney,31,2022-23,Brentford,Wolves,0.0,1785.234741,1720.871338,FWD,2.0,6.032191
5,Callum-Wilson,31,2022-23,Newcastle Utd,Aston Villa,0.0,1879.303101,1804.471680,FWD,1.0,5.738616
33,Ollie-Watkins,31,2022-23,Aston Villa,Newcastle Utd,1.0,1804.471680,1879.303101,FWD,16.0,5.456153
0,Alexander-Isak,31,2022-23,Newcastle Utd,Aston Villa,0.0,1879.303101,1804.471680,FWD,2.0,5.115699
19,Gabriel-Jesus,31,2022-23,Arsenal,West Ham,0.0,1957.524048,1754.553955,FWD,7.0,3.957874
6,Carlos-Vinicius,31,2022-23,Fulham,Everton,0.0,1713.854858,1690.919678,FWD,1.0,3.514711
3,Brennan-Johnson,31,2022-23,Nott'ham Forest,Manchester Utd,1.0,1638.596069,1882.111816,FWD,2.0,3.403676
14,Eddie-Nketiah,31,2022-23,Arsenal,West Ham,0.0,1957.524048,1754.553955,FWD,1.0,3.363196


# Feature importance and influence

In [55]:
# plot_importance(model)

In [56]:
# fig, ax = plt.subplots(figsize=(16, 8))
# plot_partial_dependence(model, X_train, features, ax=ax)

In [57]:
# explainer = shap.Explainer(model.predict, X_test)
# shap_values = explainer(X_test)

In [58]:
# shap.plots.bar(shap_values)

In [59]:
# shap.plots.beeswarm(shap_values)

In [60]:
# shap.plots.bar(shap_values[218], max_display=15)

In [61]:
# get_top_performer_names(df, 20, 50)

In [62]:
# df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name"].isin(get_top_performer_names(df, 50, 50)))]